In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
# 用来解决一些奇怪的错误

from pathlib import Path

import rasterio
from utils.data_io import get_value_from_raster

from utils.data_io import get_csv
from utils.property import ICESAT2Properties

In [2]:
data = get_csv(Path("keypoints_center.csv"))
# 提取部分数据
ds = data[
    [
        "Latitude (deg)",
        "Longitude (deg)",
        "UTM Easting (m)",
        "UTM Northing (m)",
        "Height (m MSL)",
        "Height (m HAE)",
        ICESAT2Properties.AlongTrack.value,
    ]
]

In [3]:
from utils.data_io import reproject2


s2a_path = Path(
    "data\sentinel-2\subset_1_of_S2A_MSIL2A_20250106T031121_N0511_R075_T49QCD_20250106T061847_s2resampled.tif"
)

s2a_wgs84_path = Path(
    "data\sentinel-2\subset_1_of_S2A_MSIL2A_20250106T031121_N0511_R075_T49QCD_20250106T061847_s2resampled_wgs84.tif"
)

# reproject the raster to WGS84
try:
    reproject2(s2a_path, s2a_wgs84_path, epsg=4326)
except FileExistsError:
    print(f"target already exists. Skipping reprojection.")


with rasterio.open(s2a_wgs84_path, mode="r") as src:
    raster_data = get_value_from_raster(
        src, ds["Longitude (deg)"].values, ds["Latitude (deg)"].values, index=0
    )


print(raster_data.shape)

target already exists. Skipping reprojection.
EPSG:4326
(12, 119)


In [ ]:
dem_path = Path("data/dem/GMRT_resample.tif")

real_height = []
with rasterio.open(dem_path, mode="r") as src:
    values = get_value_from_raster(
        src, ds["Longitude (deg)"].values, ds["Latitude (deg)"].values, index=1
    )
    real_height.extend(values)

EPSG:4326
lon:109.23270561662244
lat:21.40047192743329
[np.float32(-1.003813), np.float32(-1.0040649), np.float32(-1.0040954)]


# 拼接数据集

In [ ]:
from utils.data_io import save_csv


add_data = {}

for i in range(raster_data.shape[0]):
    "第9波段是8A，没有第10波段"
    i = i + 1
    if i < 9:
        band_num = str(i)
    elif i == 9:
        band_num = "8A"
    elif i == 10:
        band_num = "9"
    elif i > 10:
        band_num = str(i)
    else:
        raise ValueError("Unexpected band index")

    add_data["B" + band_num] = raster_data[i - 1, :]

add_data["real_height"] = real_height

print(f"add_data index:{list(add_data.keys())}")

ml_data = ds.assign(**add_data)
print(ml_data.columns)

dp = Path("dataset.csv")

print(f"shape of ml_data: {ml_data.shape}")
save_csv(ml_data, data_path=dp, backup=True, overwrite=True)

add_data index:['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'real_height']
Index(['Latitude (deg)', 'Longitude (deg)', 'UTM Easting (m)',
       'UTM Northing (m)', 'Height (m MSL)', 'Height (m HAE)',
       'Along-Track (m)', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8',
       'B8A', 'B9', 'B11', 'B12', 'real_height'],
      dtype='object')
shape of ml_data: (119, 20)
Save dataset_train.csv successfully!
try to backup dataset_train.csv to e:\Documents\CourseStudy\GraduationProject\program\log
Backup dataset_train.csv to e:\Documents\CourseStudy\GraduationProject\program\log\dataset_train_20250526190048.csv


True